In [21]:
import os

# Define the dataset path
dataset_path = '/kaggle/input/diamos-plant-dataset/Pear/leaves'

# Check if the dataset path exists
if not os.path.exists(dataset_path):
    print(f"Error: Dataset path '{dataset_path}' does not exist.")
else:
    # Count the number of files in each subfolder (Pear and leaves)
    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            num_files = len(os.listdir(folder_path))
            print(f"Number of files in '{folder}' folder: {num_files}")

Number of files in 'curl' folder: 65
Number of files in 'healthy' folder: 43
Number of files in 'spot' folder: 1768
Number of files in 'slug' folder: 4050


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image

In [4]:
# Define the path to your dataset
dataset_path = '/kaggle/input/diamos-plant-dataset/Pear/leaves'

In [5]:
# Define transformations for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize image to 224x224
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize image
])

In [6]:
# Function to check if image is valid
def is_valid_image(image_path):
    try:
        # Attempt to open and validate the image
        Image.open(image_path).convert('RGB')
        return True
    except (FileNotFoundError, OSError, ValueError):
        return False

In [7]:
# Custom dataset class with error handling
class CustomImageFolder(ImageFolder):
    def __init__(self, root, transform=None):
        super().__init__(root, transform=transform)

    def __getitem__(self, index):
        # Get image path and label from original ImageFolder
        path, target = self.samples[index]

        # Check if image is valid
        if not is_valid_image(path):
            # Handle invalid image (e.g., return a placeholder image or skip)
            # For simplicity, returning a placeholder image and an out-of-bounds target
            invalid_image = torch.zeros((3, 224, 224))  # Placeholder image tensor
            return invalid_image, len(self.classes)  # Return an out-of-bounds target

        # Load and transform the image
        image = self.loader(path)
        if self.transform is not None:
            image = self.transform(image)

        return image, target

In [12]:
# Load the custom dataset
dataset = CustomImageFolder(dataset_path, transform=transform)

In [13]:
# Define the DataLoader
batch_size = 32
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [14]:
# Define the Channel Shuffle module
class ChannelShuffle(nn.Module):
    def __init__(self, groups):
        super(ChannelShuffle, self).__init__()
        self.groups = groups

    def forward(self, x):
        batchsize, num_channels, height, width = x.data.size()
        channels_per_group = num_channels // self.groups
        # Reshape
        x = x.view(batchsize, self.groups, channels_per_group, height, width)
        # Transpose
        x = torch.transpose(x, 1, 2).contiguous()
        # Flatten
        x = x.view(batchsize, -1, height, width)
        return x

In [15]:
# Define the RCSA module
class RCSA(nn.Module):
    def __init__(self, in_channels, reduction=16, groups=4):
        super(RCSA, self).__init__()
        self.groups = groups
        self.channel_shuffle = ChannelShuffle(groups)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv1 = nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        residual = x
        # Channel Shuffle
        x = self.channel_shuffle(x)
        # Squeeze and Excitation
        x = self.avg_pool(x)
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.sigmoid(x)
        # Scale
        x = residual * x
        return x

In [16]:
# Define the CAM module
class CAM(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(CAM, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1, bias=False)
        self.conv2 = nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1, bias=False)
        self.sigmoid = nn.Sigmoid()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)

    def forward(self, x):
        avg_out = self.conv2(self.conv1(self.avg_pool(x)))
        max_out = self.conv2(self.conv1(self.max_pool(x)))
        out = avg_out + max_out
        return x * self.sigmoid(out)

In [17]:
# Define your custom CNN architecture with RCSA and CAM
class CustomCNN(nn.Module):
    def __init__(self, num_classes):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.rcsa1 = RCSA(16)
        self.cam1 = CAM(16)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.rcsa2 = RCSA(32)
        self.cam2 = CAM(32)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)  # Adjust input size based on your image dimensions after pooling
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.rcsa1(x)
        x = self.cam1(x)
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.rcsa2(x)
        x = self.cam2(x)
        x = self.pool2(x)
        x = x.view(-1, 32 * 56 * 56)  # Adjust based on your image dimensions after pooling
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [18]:
# Initialize the model
model = CustomCNN(num_classes=len(dataset.classes))

In [19]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=len(dataset.classes))  # Ignore the out-of-bounds target
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [20]:
# Function to calculate accuracy
def calculate_accuracy(outputs, labels):
    _, predicted = torch.max(outputs, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total * 100
    return accuracy

In [ ]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, (images, labels) in enumerate(data_loader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(data_loader)}], Loss: {running_loss / 10:.4f}')
            running_loss = 0.0

    # Print accuracy after each epoch
    epoch_accuracy = total_correct / total_samples * 100
    print(f'Epoch [{epoch+1}/{num_epochs}], Accuracy: {epoch_accuracy:.2f}%')

print('Finished Training.')

# Save the trained model if needed
torch.save(model.state_dict(), 'custom_cnn_with_rcsa_cam.pth')